In [1]:
%pylab inline
import sqlite3,copy,gc
import pandas as pd
import statsmodels.formula.api as sm
import numpy as np

import matplotlib.pyplot as plt 
from statsmodels.tsa.stattools import adfuller
import multiprocessing as mp
import time

Populating the interactive namespace from numpy and matplotlib


In [2]:
%%HTML
<button onclick="$('.input, .prompt, .output_stderr, .output_error, .output_result').toggle();">Toggle Code</button>

In [3]:
def InitalSqliteConnection(target):

    if target != 0:
        conn = sqlite3.connect(target)
    else:
        conn = sqlite3.connect(":memory:")

    return conn

def CollectedTableName(conn):
    #collect the name of table
    cur = conn.cursor()
    cur.execute(
        "select name from sqlite_master where type='table' order by name")
    return [table[0].encode("utf-8") for table in cur.fetchall()]

def InitialSelectData(databasefilepath):
    
    conn = InitalSqliteConnection(databasefilepath)
    tables = CollectedTableName(conn)
    return tables,conn
def ReturnRate(onelist):
    
    end=copy.deepcopy(onelist)
    onelist.pop()
    del end[0]
    temp = [bb-aa for (aa,bb) in zip(onelist,end)]
    temp.insert(0,"NAN")
    return temp
def DetectStatus(mas,prices):
    temp =zip(mas,prices)
    buy=[]
    for ma,price in temp:
        if price < ma:
            buy.append(1)
        else:
            buy.append(0)
    return buy
    

def variable2dummy(allll):
    temp=[]
    for al in allll:
        if al >0:
            temp.append(1)
        else:
            temp.append(0)
    return temp 
def GoogleSearchLevel(dfseries,df):
    level=[]
    for d in list(dfseries):
        if d<2:
            level.append(0)
      
        elif d < 25 and d>=2:
            level.append(1)
        elif d>=25 and d<50:
            level.append(2)
        elif d>=50 and d<75:
            level.append(3)
        else:
            level.append(4)
    df['GoogleSearch_level']=level
    return df
def NewsHeatLevel(dfseries,df):
    #if d < 0.20:
        #df['NewsHeat_level']=1
    #elif d>=1 and d<2:
    #    df['NewsHeat_level']=2
    #elif d>=2 and d<3:
    #    df['NewsHeat_level']=3
    #else:
        #df['NewsHeat_level']=2
    df['NewsHeat_level']=list(dfseries)
    return df
def CreateLevel(df,printout=False):
    replaceAbnormalValue(df['GoogleSearch'])
    replaceAbnormalValue(df['NEWS_HEAT_READ_DAVG'])
    
    if df['GoogleSearch'].describe()[1]>0:
        
        if printout!=False:
            print "Do ols regression for GoogleSearch"
        
            df['GoogleSearch'].hist(bins=20,alpha=0.5, color='k', normed=True)
            df['GoogleSearch'].plot(kind='kde', style='k--')
        else:
            pass
    else:
        pass
    
    df=GoogleSearchLevel(df['GoogleSearch'],df)
    
    if df['NEWS_HEAT_READ_DAVG'].describe()[1]>0:
        
        if printout!=False:
            print "do ols regression for NEWS_HEAT_READ_DAVG"
            df['NEWS_HEAT_READ_DAVG'].hist(alpha=0.5, color='red', normed=True)
            df['NEWS_HEAT_READ_DAVG'].plot(kind='kde', style='k--')
        else:
            pass
    
    else:
        pass
    if printout!=False:
        print "Mean of GoogleSearch for %s:"%table,df['GoogleSearch'].describe()[1]
        print "Mean of NEWS_HEAT_READ_DAVG for %s:"%table,df['NEWS_HEAT_READ_DAVG'].describe()[1]
    df=NewsHeatLevel(df['NEWS_HEAT_READ_DAVG'],df)
    return df
def replaceAbnormalValue(df):
    l =[s for s in set(df) if s<0]
    
    if len(l) != 0:
        temp=[0]*len(l)
        t =dict(zip(l,temp))
        df.replace(t,inplace=True)
    else:
        pass
def SelectDataforStats(df,table):
    temp_df =df[['PRC','Return','MARE30','MAP','GoogleSearch_level','NewsHeat_level']]
    a =dict(zip(['PRC','Return','MARE30','MAP','GoogleSearch_level','NewsHeat_level'],list(mean(temp_df))))
    a.update({"ticker":table})
    return a
def drawGraph(info_pd,target_variable,limited_variable):
    if len(set(info_pd[limited_variable]))>1:
        tests =[(s,info_pd[target_variable][info_pd[limited_variable]==s]) for s in set(info_pd[limited_variable])]
    else:
        s =set(info_pd[limited_variable])[0]
        tests =[(s,info_pd[target_variable][info_pd[limited_variable]==s])]
    for s,test in tests:
        if len(test)<=2:
            pass
        else:
            test.hist(bins=20,alpha=0.5, color='red', normed=True)
            test.plot(kind='kde', title='%s Under %s level =%s'%(target_variable,limited_variable,s),style='k--',label='Density Distribution',legend=True)
            plt.grid(True)  
            plt.show()
        
        
def adftest(series_test,printout=False):
    X = series_test.values
    result = adfuller(X)
    if printout !=False:
        print('ADF Statistic: %f' % result[0])
        print('p-value: %f' % result[1])
        print('Critical Values:')
        for key, value in result[4].items():
            print('\t%s: %.3f' % (key, value))
        if result[1] <0.05:
            print "Statistical Stationary"
        else:
            print "Non-Stationary"
    else:
        pass
    return (result[0],result[1])

def TestADFandProcessOther(df,printout=False,othertype="Diff",process=[]):
    (adf,p_value)=adftest(df,False)

    if p_value>=0.05:
        #print p_value
        if othertype=="Diff":
            process.append("Diff")
            (adf,p_value,process) = TestADFandProcessOther(df.diff(1)[1:],printout,"Diff",process)
        else:
            process.append("Log")
            (adf,p_value,process) = TestADFandProcessOther(log(df),printout,"Log",process)
    else:
        pass
    return (adf,p_value,process)

def Diff2DateFrame(df,n=1):
    return df.diff(n)[n:]
def Log2DateFrame(df):
    return log(df)
def DetectADFtestandReturnVarlist(df,Need_test):
    var_list=[]
    for var in Need_test:
        adf_result =TestADFandProcessOther(df[var],process=[])
        if len(adf_result[2])==0:
            var_list.append(var)
        else:

            df['%sDiff%s'%(var,len(adf_result[2]))]=Diff2DateFrame(df['%s'%var],len(adf_result[2]))
            var_list.append('%sDiff%s'%(var,len(adf_result[2])))

    #print var,adf_result[2]
    #print adf_result
    #price_adf,price_adf_p=adftest(df['PRC'].diff(1)[1:],True)
    #google_adf,google_adf_p=adftest(df['GoogleSearch'],True)
    #adf_result =TestADFandProcessOther(df['GoogleSearch'],process=[])
    #print adf_result[2] 
    return var_list
def ReturnOlsCMD(x_var_list,x,y_var_list):
    xa=copy.deepcopy(x_var_list)
    ya=copy.deepcopy(y_var_list)
    xa.append(x)
    ya.extend(xa)
    return ya
def OlsTestProcess(NewsHeat,GoogleSearch,df):
    result_bloomberg = sm.ols(formula="%s ~ %s+%s+%s+%s+%s"%(NewsHeat[0],NewsHeat[1],NewsHeat[2],NewsHeat[3],NewsHeat[4],NewsHeat[5]), data=df).fit()
    result_googlesearch = sm.ols(formula="%s ~ %s+%s+%s+%s+%s"%(GoogleSearch[0],GoogleSearch[1],GoogleSearch[2],GoogleSearch[3],GoogleSearch[4],GoogleSearch[5]), data=df).fit()
    #result_googlesearch = sm.ols(formula="MAP ~VOL+MktRF+SMB+HML+GoogleSearch", data=df).fit()
    #print result_bloomberg.params
    #print result_bloomberg.summary()

    #print result_googlesearch.params,result_googlesearch.pvalues
    #print result_bloomberg.params
    #print result_bloomberg.pvalues[-1]
    #print result_bloomberg.f_pvalue
    #print result_bloomberg.summary()
    return result_bloomberg,result_googlesearch
def varlist(df,x_Need_test,y_Need_test,x_Attention_Need_test):
    
    x_var_list=DetectADFtestandReturnVarlist(df,x_Need_test)
    #varlist.extend(x_var_list)
    y_var_list=DetectADFtestandReturnVarlist(df,y_Need_test)
    #varlist.extend(y_var_list)
    x_Attention_var_list=DetectADFtestandReturnVarlist(df,x_Attention_Need_test)
    
    #x_var_list,y_var_list
    #print x_var_list,y_var_list,x_Attention_var_list,TestADFandProcessOther(df['NEWS_HEAT_READ_DAVG'],process=[])[2]
    
    #print DetectADFtestandReturnVarlist(df,x_Need_test)
    return x_var_list,y_var_list,x_Attention_var_list
def pdmerge(a,b):
    return pd.merge(a,b,how='outer',sort=False)
def pdjoin(a,*b):
    return a.join(*b,on="Date",how='outer',sort=False)
def mergeDateFrame(*b):
    return reduce(pdmerge,*b)
def RefineDataFrame(rr,paramslist,sort='Date',drop =False):
    for key,value in paramslist.items():
        rr[key] = DetecteAbnormalValue(rr[key],nagetive=value[0],unwantedstr=value[1],dtype=value[2],fillna=value[3])
    if sort != False:
        if sort in list(rr.columns):
            rr=rr.sort([sort]).reset_index()
            rr.drop('index',axis=1, inplace=True)
        else:
            pass
    else:
        pass
    if drop !=False:
        rr.dropna()
    return rr
def DetecteAbnormalValue(dfseries,nagetive='fillzero',unwantedstr=0,dtype=False,fillna="dontfill"):
    
    #unwanted str
    if unwantedstr !=False:
    
        dfseries=pd.to_numeric(dfseries,errors='coerce')
        
    else:
        pass
    #negative value
    if nagetive =='fillzero':
        abnormal_nagtive=[i for i in set(dfseries) if i <0]
        if abnormal_nagtive != []:
            dfseries.replace(abnormal_nagtive,0)
        else:
            pass
        
    elif nagetive == 'abs':
        dfseries = abs(dfseries)
        
    else:
        pass
    
    if fillna == "dontfill":
        pass
        
    elif fillna=='mean':
        dfseries=dfseries.fillna(dfseries.mean())
    elif fillna=='max':
        dfseries=dfseries.fillna(dfseries.max())
    else:
        dfseries=dfseries.fillna(fillna)
    
    
    
    if dtype != False:
        dfseries=pd.to_numeric(dfseries,errors='coerce')
        dfseries=dfseries.astype(dtype)
    else:
        pass
    return dfseries

In [6]:
table_shares,conn_shares =InitialSelectData('/Users/sn0wfree/Documents/python_projects/data_collection/API/Share/wrds/sharesWRDS.db')

In [120]:
print table_shares[2]
rr =pd.read_sql_query("select Date, PRC,RET,VOL,SICCD from %s"%table_shares[2],conn_shares).drop_duplicates()
rr['SICCD']=pd.to_numeric(rr['SICCD']).astype(np.int64)
rr['Date'] =rr['Date'].astype(object)
#detect abnormal value:

        
    
    
abnormal=[i for i in set(rr['VOL']) if i <0]
if abnormal != []:
    rr['VOL'].replace(abnormal,0)
else:
    pass
rr['PRC'] = abs(rr['PRC'])
rr.fillna(0)
rr['RET'] =pd.to_numeric(rr['RET'],errors='coerce')


AAC


In [7]:
rr =pd.read_sql_query("select Date, PRC,RET,VOL,SICCD,TICKER from %s"%table_shares[20],conn_shares).drop_duplicates()

paramslist={'Date':(abs,1,np.int64,'fillzero'),
         'PRC':(abs,1,np.float64,'mean'),
         'RET':(0,1,np.float64,'mean'),
         'VOL':(abs,1,np.int64,'mean'),
         'SICCD':(abs,1,np.int64,'mean')
        }
#rr =RefineDataFrame(rr,paramslist,'Date',drop=1)

rr


,PERMNO,Date,SICCD,TICKER,COMNAM,SECSTAT,CUSIP,DIVAMT,FACPR,FACSHR,...,ASK,CFACPR,CFACSHR,OPENPRC,RETX,vwretd,vwretx,ewretd,ewretx,sprtrn
0,90825,20050811,3579.0,ABD,A C C O BRANDS CORP,W,00081T10,None,None,None,...,NaN,NaN,NaN,NaN,None,0.007424,0.007279,0.006125,0.005820,0.007062
1,90825,20050817,3579.0,ABD,A C C O BRANDS CORP,R,00081T10,None,None,None,...,24.85,1.0,1.0,22.90,C,-0.000623,-0.000831,-0.000445,-0.000518,0.000738
2,90825,20050818,3579.0,ABD,A C C O BRANDS CORP,R,00081T10,None,None,None,...,26.33,1.0,1.0,24.30,0.059131,-0.002087,-0.002121,-0.003425,-0.003458,-0.001000
3,90825,20050819,3579.0,ABD,A C C O BRANDS CORP,R,00081T10,None,None,None,...,26.76,1.0,1.0,25.90,0.015951,0.001966,0.001899,0.002110,0.002085,0.000566
4,90825,20050822,3579.0,ABD,A C C O BRANDS CORP,R,00081T10,None,None,None,...,27.00,1.0,1.0,26.30,0.011215,0.002571,0.002559,0.003189,0.003157,0.001656
5,90825,20050823,3579.0,ABD,A C C O BRANDS CORP,R,00081T10,None,None,None,...,25.80,1.0,1.0,26.90,-0.040665,-0.002845,-0.002886,-0.000552,-0.000601,-0.003405
6,90825,20050824,3579.0,ABD,A C C O BRANDS CORP,R,00081T10,None,None,None,...,24.85,1.0,1.0,24.80,-0.050482,-0.004395,-0.004450,-0.001046,-0.001098,-0.006554
7,90825,20050825,3579.0,ABD,A C C O BRANDS CORP,R,00081T10,None,None,None,...,23.81,1.0,1.0,23.90,-0.034091,0.002225,0.002212,0.002506,0.002489,0.002315
8,90825,20050826,3579.0,ABD,A C C O BRANDS CORP,R,00081T10,None,None,None,...,24.02,1.0,1.0,23.60,0.010504,-0.006719,-0.006733,-0.005618,-0.005636,-0.006013
9,90825,20050829,3579.0,ABD,A C C O BRANDS CORP,R,00081T10,None,None,None,...,26.00,1.0,1.0,24.00,0.110187,0.006148,0.006042,0.003948,0.003885,0.005958


rr.columns

In [157]:
paramslist={'Date':(abs,1,np.int64,'fillzero'),
         'PRC':(abs,1,np.float64,'mean'),
         'RET':(0,1,np.float64,'mean'),
         'VOL':(abs,1,np.int64,'mean'),
         'SICCD':(abs,1,np.int64,'mean'),
        }
def RefineDataFrame(rr,paramslist,sort='date'):
    for key,value in paramslist.items():
        rr[key] = DetecteAbnormalValue(rr[key],nagetive=value[0],unwantedstr=value[1],dtype=value[2],fillna=value[3])
    if sort != False:
        if sort in list(rr.columns):
            rr=rr.sort([sort]).reset_index()
            rr.drop('index',axis=1, inplace=True)
        else:
            pass
    else:
        pass
    return rr
def RefineDataFrame(rr):
    rr['Date'] = DetecteAbnormalValue(rr['Date'],nagetive='abs',unwantedstr=1,dtype=np.int64)
    rr['PRC'] = DetecteAbnormalValue(rr['PRC'],nagetive='abs',unwantedstr=1,dtype=np.float64,fillna='mean')
    rr['RET'] = DetecteAbnormalValue(rr['RET'],nagetive=0,unwantedstr=1,dtype=np.float64,fillna='mean')
    rr['VOL'] = DetecteAbnormalValue(rr['VOL'],nagetive='abs',unwantedstr=1,dtype=np.int64)
    rr['SICCD'] = DetecteAbnormalValue(rr['SICCD'],nagetive='abs',unwantedstr=1,dtype=np.int64)

    rr =rr.sort(['Date']).reset_index()
    rr.drop('index',axis=1, inplace=True)
    return rr

In [119]:
set(rr.columns)

{'Date', 'PRC', 'RET', 'SICCD', 'VOL'}

In [155]:
w =pd.DataFrame({'a':[1,2],"b":[2,3],"c":['a',1]})

w['c']=pd.to_numeric(w['c'],errors='coerce').fillna(0).astype(np.int64)


In [4]:
share2='/Users/sn0wfree/Documents/python_projects/data_collection/API/Share/share2.csv'
share2_pd =pd.read_csv(share2,low_memory=False)


In [14]:
len(set(share2_pd['TICKER']))

3519

In [ ]:
share2_pd.columns

In [5]:
#{data series, how to negative value, whether want str, transform into dtype, use XXX fill NA value}
paramslist={
         'PRC':(abs,1,np.float64,'mean'),
         'RET':(0,1,np.float64,'mean'),
         'VOL':(abs,1,np.int64,'mean'),
         'SICCD':(abs,1,np.int64,'mean'),
            
            'SHROUT':(abs,0,np.int64,'mean')}
share2_pd =RefineDataFrame(share2_pd,paramslist,sort='date',drop=1)


/Users/sn0wfree/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:237: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,PERMNO,date,SICCD,TICKER,PRC,VOL,RET,SHROUT
0,10025,20040102,3070,AEPI,9.650000,12400,-0.009139,8177
1,38762,20040102,4931,NI,21.660000,757100,-0.012762,262465
2,88837,20040102,3812,GRMN,53.690000,467034,-0.014501,108166
3,37161,20040102,4911,TE,14.390000,899100,-0.001388,176669
4,11382,20040102,5311,SKS,14.820000,362800,-0.014628,136557
5,85710,20040102,3670,POWI,33.360000,242982,-0.002989,30408
6,10180,20040102,2830,NaN,34.403508,1602022,0.000670,19569
7,89262,20040102,6035,WFD,23.400000,14200,-0.015152,10522
8,90352,20040102,4922,DYN,4.550000,5544200,0.063084,280101
9,76841,20040102,2830,BIIB,36.940000,2698977,0.006539,330410


In [6]:
share2_pd['MarketValue'] = share2_pd['PRC']*share2_pd['SHROUT']


,PERMNO,date,SICCD,TICKER,PRC,VOL,RET,SHROUT,MarketValue
0,10025,20040102,3070,AEPI,9.650000,12400,-0.009139,8177,7.890805e+04
1,38762,20040102,4931,NI,21.660000,757100,-0.012762,262465,5.684992e+06
2,88837,20040102,3812,GRMN,53.690000,467034,-0.014501,108166,5.807433e+06
3,37161,20040102,4911,TE,14.390000,899100,-0.001388,176669,2.542267e+06
4,11382,20040102,5311,SKS,14.820000,362800,-0.014628,136557,2.023775e+06
5,85710,20040102,3670,POWI,33.360000,242982,-0.002989,30408,1.014411e+06
6,10180,20040102,2830,NaN,34.403508,1602022,0.000670,19569,6.732422e+05
7,89262,20040102,6035,WFD,23.400000,14200,-0.015152,10522,2.462148e+05
8,90352,20040102,4922,DYN,4.550000,5544200,0.063084,280101,1.274460e+06
9,76841,20040102,2830,BIIB,36.940000,2698977,0.006539,330410,1.220535e+07


In [ ]:
conn =sqlite3.connect('shares.db')
share2_pd.to_sql('share',conn,if_exists='replace', index=False)

In [ ]:
'date':(abs,1,np.int64,'fillzero'),
'TICKER':(abs,0,np.object,'mean'),